<a href="https://colab.research.google.com/github/alishachandni/Cloud/blob/main/Retinal_Imaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths
dataset_path = '/content/drive/MyDrive/Final Project/Retina/retina'

In [ ]:
# Hyperparameters
batch_size = 32
image_size = 128  # Adjusted for higher resolution
latent_dim = 100
epochs = 20
lr = 0.0002
beta1 = 0.5

In [ ]:
# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

In [ ]:
# Load dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# LSTM-GAN Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.lstm = nn.LSTM(latent_dim, 256, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, image_size * image_size * 3),  # 3 channels for RGB
            nn.Tanh()
        )

    def forward(self, z):
        lstm_out, _ = self.lstm(z.unsqueeze(1))
        img = self.fc(lstm_out[:, -1, :])
        return img.view(img.size(0), 3, image_size, image_size)

In [ ]:
# LSTM-GAN Discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.lstm = nn.LSTM(image_size * image_size * 3, 256, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), 1, -1)
        lstm_out, _ = self.lstm(img_flat)
        validity = self.fc(lstm_out[:, -1, :])
        return validity

In [ ]:
# Initialize models
generator = Generator().cuda()
discriminator = Discriminator().cuda()

In [ ]:
# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Tracking losses
d_losses, g_losses = [], []
y_true, y_scores = [], []

In [ ]:
# Training loop
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Real and fake labels
        real_labels = torch.ones(imgs.size(0), 1).cuda()
        fake_labels = torch.zeros(imgs.size(0), 1).cuda()

        # Train Discriminator
        imgs = imgs.cuda()
        optimizer_D.zero_grad()
        outputs = discriminator(imgs)
        d_loss_real = adversarial_loss(outputs, real_labels)

        z = torch.randn(imgs.size(0), latent_dim).cuda()
        fake_imgs = generator(z)
        outputs = discriminator(fake_imgs.detach())
        d_loss_fake = adversarial_loss(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()
        d_losses.append(d_loss.item())

        # Train Generator
        optimizer_G.zero_grad()
        outputs = discriminator(fake_imgs)
        g_loss = adversarial_loss(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()
        g_losses.append(g_loss.item())

        y_true.extend(real_labels.cpu().detach().numpy())
        y_scores.extend(outputs.cpu().detach().numpy())

        # Print progress
        if i % 50 == 0:
            print(f"[Epoch {epoch+1}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save generated samples
    if (epoch + 1) % 10 == 0:
        save_image(fake_imgs.data[:25], f"/content/drive/MyDrive/Final Project/generated_retinal_samples_epoch_{epoch+1}.png", nrow=5, normalize=True)

[Epoch 1/20] [Batch 0/100] [D loss: 1.4012980461120605] [G loss: 0.7885439395904541]
[Epoch 1/20] [Batch 50/100] [D loss: 0.39675450325012207] [G loss: 1.2172175645828247]
[Epoch 2/20] [Batch 0/100] [D loss: 0.3868003785610199] [G loss: 1.2638144493103027]
[Epoch 2/20] [Batch 50/100] [D loss: 0.3250505328178406] [G loss: 1.4062888622283936]
[Epoch 3/20] [Batch 0/100] [D loss: 0.23067089915275574] [G loss: 1.6374642848968506]
[Epoch 3/20] [Batch 50/100] [D loss: 0.19621486961841583] [G loss: 1.8427133560180664]
[Epoch 4/20] [Batch 0/100] [D loss: 0.22486773133277893] [G loss: 1.6726797819137573]


In [ ]:
# Compute sensitivity and specificity
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(y_true, y_scores)

In [ ]:
# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Plot Precision-Recall Curve
plt.figure()
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [ ]:
# Plot Losses
plt.figure()
plt.plot(d_losses, label='Discriminator Loss')
plt.plot(g_losses, label='Generator Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()
plt.show()

In [ ]:
print("Training complete. Check generated samples and graphs in Google Drive.")